# Quantum Dynamic Pricing
#### We employ the OLA CABS dataset to forecast taxi fare rates utilizing a Quantum Neural Network. 
#### The same code can be use on any dynamic pricing dataset for determining price of product/service offered.

## Installing required Libraries 

In [ ]:
!pip install tensorflow==2.7.0
!pip install tensorflow-quantum==0.7.2

## Exploring and Processing Dataset

In [3]:
import pandas as pd
data = pd.read_csv('../input/ola-dynamic/data.csv')
data

,Unnamed: 0,ID,vendor+AF8-id,pickup+AF8-loc,drop+AF8-loc,driver+AF8-tip,mta+AF8-tax,distance,pickup+AF8-time,drop+AF8-time,num+AF8-passengers,toll+AF8-amount,payment+AF8-method,rate+AF8-code,stored+AF8-flag,extra+AF8-charges,improvement+AF8-charge,total+AF8-amount
0,0,0,1,170.0,233.0,1.83,0.5,0.70,4/4/2017 17:59,4/4/2017 18:05,1.0,0.0,1.0,1.0,N,1.0,0.3,9.13
1,1,1,2,151.0,243.0,3.56,0.5,4.64,4/3/2017 19:03,4/3/2017 19:20,1.0,0.0,1.0,1.0,N,1.0,0.3,21.36
2,2,2,2,68.0,90.0,1.50,0.5,1.29,4/3/2017 15:06,4/3/2017 15:12,2.0,0.0,1.0,1.0,N,0.0,0.3,8.80
3,3,3,2,142.0,234.0,1.50,0.5,2.74,4/4/2017 8:10,4/4/2017 8:27,1.0,0.0,1.0,1.0,N,0.0,0.3,14.80
4,4,4,2,238.0,238.0,0.00,0.5,0.45,4/5/2017 14:02,4/5/2017 14:05,6.0,0.0,2.0,1.0,N,0.0,0.3,4.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,14995,2,41.0,238.0,1.00,0.5,2.11,4/3/2017 7:01,4/3/2017 7:08,1.0,0.0,1.0,1.0,N,0.0,0.3,9.80
14996,14996,14996,1,237.0,161.0,1.00,0.5,1.10,4/6/2017 8:01,4/6/2017 8:13,1.0,0.0,1.0,1.0,N,0.0,0.3,10.80
14997,14997,14997,2,264.0,264.0,0.00,0.5,1.00,4/5/2017 17:13,4/5/2017 17:22,5.0,0.0,2.0,1.0,N,1.0,0.3,8.30
14998,14998,14998,2,140.0,50.0,2.00,0.5,4.38,4/3/2017 20:09,4/3/2017 20:30,1.0,0.0,1.0,1.0,N,0.5,0.3,20.80


In [4]:
# Making pickup time and drop time columns use able for training the Quantum Neural Network
data['pickup+AF8-time']=pd.to_datetime(data['pickup+AF8-time'])
data['drop+AF8-time']=pd.to_datetime(data['drop+AF8-time'])

duration = (data['drop+AF8-time'] - data['pickup+AF8-time'])
data.drop(columns=['pickup+AF8-time','drop+AF8-time',],inplace=True)
timeinsec=duration.dt.total_seconds()
data['timeinsec']=timeinsec
data.loc[data['ID'] >=0, 'ID'] = 1.0

In [5]:
# Dropping columns that wont affect fare price 
drops = ['vendor+AF8-id','stored+AF8-flag','Unnamed: 0','total+AF8-amount']
x = data.drop(drops,axis=1)
y = data['total+AF8-amount']

In [6]:
# Converting x and y to numpy arrays for using MinMaxScaler 
import numpy as np
x = np.array(x)
y = np.array(y)

## Creating Quantum Neural Network and training the QNN 
#### Here we use Tensorflow Quantum and Cirq for creating the neural network. Other libraries such as Pennylane and Qiskit can also be used.  

#### 1. Data Preprocessing
The initial code snippet normalizes the input data and rescales the target variable. 

In [ ]:
x = MinMaxScaler().fit_transform(x)
y = (y - np.min(y)) / (np.max(y) - np.min(y))

#### 2. Data Visualization
To better understand the data distribution and the performance of your model, we visualize the data before and after preprocessing.

In [ ]:
# Visualize the original and preprocessed data
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.scatter(x[:, 0], y)
plt.title("Original Data")
plt.xlabel("Feature 1")
plt.ylabel("Target")

plt.subplot(1, 2, 2)
plt.scatter(X_train[:, 0], y_train)
plt.title("Preprocessed Training Data")
plt.xlabel("Normalized Feature 1")
plt.ylabel("Normalized Target")
plt.tight_layout()
plt.show()

#### 3. Encode Data:
   The `convert_data` function converts input data into quantum circuits using Rx and Ry gates. 

In [ ]:
def convert_data(data, qubits, test=False):
    cs = []
    for i in data:
        cir = cirq.Circuit()
        for j in range(len(qubits)):
            # Customize the encoding gates as needed
            cir += cirq.rx(i[j] * np.pi).on(qubits[j])
            cir += cirq.ry(i[j] * np.pi).on(qubits[j])
            # Add other encoding gates here if desired
        cs.append(cir)
    if test:
        return tfq.convert_to_tensor([cs])
    return tfq.convert_to_tensor(cs)

#### 4. Model Circuit:
   The `model_circuit` function constructs the quantum neural network's circuit. It is possible to adjust the depth and structure of the circuit.

In [ ]:
def model_circuit(qubits, depth):
    cir = cirq.Circuit()
    num_params = depth * 2 * len(qubits)
    params = sympy.symbols("q0:%d" % num_params)
    for i in range(depth):
        cir = layer(cir, qubits, params[i * 2 * len(qubits): i * 2 * len(qubits) + 2 * len(qubits)])
        # Add additional layers or modifications here if desired
    return cir

#### 5. Model Training:
   The provided code trains the quantum neural network using the `fit` function. Try to experiment with different hyperparameters, such as the number of epochs and batch size, to optimize the model's performance.

In [ ]:
# Train the quantum neural network
epochs = 50  # Increase the number of epochs for better convergence
batch_size = 32

v_history = vqc.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[callback]
)

#### 6. Plot Training History:
   Now, visualize the training and validation loss over epochs to monitor the model's performance and convergence.

In [ ]:
# Plot training history
plt.figure(figsize=(10, 5))
plt.plot(v_history.history['loss'], label='Train Loss')
plt.plot(v_history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

#### 7. Predictions:
   After training, let's can make predictions using the trained model and evaluate its performance.

In [ ]:
# Make predictions
y_pred = vqc.predict(X_test)

In [ ]:
# Evaluate performance 
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

#### 8. Further Improvements:
   - Experiment with different architectures, such as varying the number of qubits, depth of the circuit, and layers, to find the best configuration for your problem.
   - Perform hyperparameter tuning to optimize the model's performance.
   - Consider using other quantum layers and differentiators provided by TFQ for better results.
   - Explore different preprocessing techniques and feature engineering to improve data representation.
